In [1]:
# Install Gradio
!pip install -q gradio

# Imports
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import cv2
import random
from PIL import Image
import gradio as gr

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5
CLASSES = ['Neutrophil', 'Eosinophil', 'Basophil', 'Lymphocyte', 'Monocyte']
NORMAL_CLASSES = ['Neutrophil', 'Lymphocyte']
NUM_CLASSES = len(CLASSES)

# Step 1: Create synthetic blood image dataset
def create_blood_dataset():
    for split in ['train', 'val', 'test']:
        for cls in CLASSES:
            os.makedirs(f"blood_dataset/{split}/{cls}", exist_ok=True)

    colors = {
        'Neutrophil': [255, 0, 0],
        'Eosinophil': [0, 255, 0],
        'Basophil': [0, 0, 255],
        'Lymphocyte': [255, 255, 0],
        'Monocyte': [255, 0, 255]
    }

    for cls in CLASSES:
        for split, count in zip(['train', 'val', 'test'], [50, 15, 20]):
            for i in range(count):
                img = np.zeros((224, 224, 3), dtype=np.uint8)
                img[:, :, :] = colors[cls]
                cv2.putText(img, cls[0], (80, 130), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 3)
                cv2.imwrite(f"blood_dataset/{split}/{cls}/{cls}_{i}.jpg", img)

create_blood_dataset()

# Step 2: Load datasets
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'blood_dataset/train', target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'blood_dataset/val', target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'blood_dataset/test', target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

labels = list(train_gen.class_indices.keys())
print(f"\n✅ Classes detected: {train_gen.num_classes}")

# Step 3: Build model
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224,224,3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Train model
history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS,
                    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

# Save model
model.save('hematovision_model.h5')

# Step 5: Confusion Matrix
y_true = test_gen.classes
y_probs = model.predict(test_gen)
y_pred = np.argmax(y_probs, axis=1)
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,6))
disp = ConfusionMatrixDisplay(cm, display_labels=labels)
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.close()

# Step 6: Accuracy & Loss Plots
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label="Train")
plt.plot(history.history['val_accuracy'], label="Val")
plt.title("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label="Train")
plt.plot(history.history['val_loss'], label="Val")
plt.title("Loss")
plt.legend()

plt.tight_layout()
plt.savefig("metrics_plot.png")
plt.close()

# Step 7: Sample predictions - Mixed (Normal & Abnormal)
sample_paths = []
# 3 Normal
for cls in NORMAL_CLASSES:
    cls_path = f"blood_dataset/test/{cls}"
    img_file = random.choice(os.listdir(cls_path))
    sample_paths.append(f"{cls_path}/{img_file}")

# 3 Abnormal
for cls in set(CLASSES) - set(NORMAL_CLASSES):
    cls_path = f"blood_dataset/test/{cls}"
    img_file = random.choice(os.listdir(cls_path))
    sample_paths.append(f"{cls_path}/{img_file}")

# Step 8: Prediction function with Normal/Abnormal
def predict_and_plot(img_path):
    img = Image.open(img_path).resize(IMG_SIZE)
    img_arr = np.expand_dims(np.array(img)/255.0, axis=0)
    probs = model.predict(img_arr)[0]
    pred_idx = np.argmax(probs)
    pred_label = labels[pred_idx]
    conf = probs[pred_idx]
    status = "Normal" if pred_label in NORMAL_CLASSES else "Abnormal"

    plt.figure(figsize=(4, 6))
    plt.imshow(img)
    plt.title(f"{pred_label} ({conf:.2f}) - {status}", fontsize=12)
    plt.axis("off")
    out_path = f"pred_{os.path.basename(img_path)}"
    plt.savefig(out_path)
    plt.close()
    return out_path

# Step 9: Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🩸 Hematovision: Blood Cell Classification Demo")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 🔬 Confusion Matrix")
            gr.Image(value="confusion_matrix.png", label="Confusion Matrix")

        with gr.Column():
            gr.Markdown("### 📈 Accuracy and Loss")
            gr.Image(value="metrics_plot.png", label="Accuracy & Loss")

    gr.Markdown("### 🧪 Sample Predictions (Mixed Normal & Abnormal)")

    for path in sample_paths:
        gr.Image(value=predict_and_plot(path), label=os.path.basename(path))

demo.launch()


Found 250 images belonging to 5 classes.
Found 75 images belonging to 5 classes.
Found 100 images belonging to 5 classes.

✅ Classes detected: 5
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2159 - loss: 1.6857

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.2128 - loss: 1.6857 - val_accuracy: 0.2000 - val_loss: 1.6324
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.2001 - loss: 1.6913 - val_accuracy: 0.2000 - val_loss: 1.6203
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.1952 - loss: 1.6169 - val_accuracy: 0.2000 - val_loss: 1.6102
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.2026 - loss: 1.6084 - val_accuracy: 0.2000 - val_loss: 1.6159
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.1814 - loss: 1.6230 - val_accuracy: 0.2000 - val_loss: 1.6079


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2477270fd355aefe39.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<Figure size 600x600 with 0 Axes>